In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import requests
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from tqdm.auto import tqdm
import random

In [2]:
# 크롤링 함수
def crawl(start, end):
    for p in range(start,end):
        html = driver.page_source
        soup = bs(html, 'html.parser')
        
        reviews = soup.select('#review > ul > li')
        
        for r in reviews:
            review.append(r.select_one('.txt').text)
            star.append(r.select_one('.score_wrap_sm .num').text)

        driver.find_element(By.XPATH, '//*[@id="pagination"]/div/button['+str(p)+']').click()
        time.sleep(0.5)
        #print(soup.select('.paging .on')[0].text)

In [4]:
def make_review_crawl(url):

    driver.get(url)

    # 페이지 파싱
    html = driver.page_source
    soup = bs(html, 'html.parser')

    # '리뷰' 클릭하도록 명령
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/button[3]').click()
    
    top_text = soup.select('.score_top')[0].find('strong').text
    top_score = soup.select('.score_top .num')[0].text
    top_total=soup.select('.score_top')[0].find('p').find('b').text

    # 리뷰 총 개수
    top_total = int(soup.select('.score_top')[0].find('p').find('b').text)

    # 마지막 페이지 수 (한페이지에 리뷰 10개씩)
    last_page = top_total // 10 + 1 

    # next_page 몇 번 클릭 해야하는지 설정
    if last_page % 5 == 0:
        page_step = last_page // 5 
    else:
        page_step = last_page // 5 + 1   
    
    #print(top_total, last_page, page_step)
    step = 1
    while(page_step >= step):
        
        # 첫번째 step
        if step == 1:
            #print('step1 loop')
            crawl(2,7)
            
        # 마지막 step
        elif step == page_step:
            #print('step 마지막 loop')

                
            if top_total % 50 > 10:
                            
                if last_page % 5 == 0:
                    end_page = 7
                else: 
                    end_page =last_page % 5 + 2
                
                crawl(3, end_page)

            # 마지막 페이지
            html = driver.page_source
            soup = bs(html, 'html.parser')
            reviews = soup.select('#review > ul > li')
        
            for r in reviews:
                review.append(r.select_one('.txt').text)
                star.append(r.select_one('.score_wrap_sm .num').text)    
        # 중간 step
        else:
            #print('step 중간 loop')
            crawl(3,8)
        step += 1

In [5]:
driver = webdriver.Chrome()
# 여기어때 상품 리뷰 주소
url = "https://www.goodchoice.kr/product/detail?ano=6010"
driver.get(url)

# 페이지 파싱
html = driver.page_source
soup = bs(html, 'html.parser')

In [9]:
# 전체 상품 중에 랜덤하게 추출
random.seed(42)
idx = random.sample(range(1,70000), 1000)
idx = sorted(idx)
#print(idx)

In [ ]:
driver = webdriver.Chrome()
review = []
star = []

# 페이지 로딩 3초 대기
driver.implicitly_wait(3)
for i in tqdm(idx):
    try:
        make_review_crawl(f'https://www.goodchoice.kr/product/detail?ano={i}')
    except:
        # 상품 고유번호가 없는 경우 error 출력
        print(i, 'error')
        continue
driver.close()

In [ ]:
#df = pd.DataFrame({'review':review, 'star':star})
#df.to_csv('여기어때_리뷰_크롤링.csv', encoding = 'utf-8-sig', index=False)